In [45]:
import pandas as pd
import numpy as np

In [46]:
curr_reg = pd.read_csv('regboxscores2024-25.csv')
curr_adv = pd.read_csv('advboxscores2024-25.csv')

In [47]:
curr_reg.rename(columns={'Match Up': 'MATCHUP', 'Game Date': 'GAME_DATE'}, inplace=True)

In [48]:
curr_adv.rename(columns={'Match\xa0Up': 'MATCHUP', 'Game\xa0Date': 'GAME_DATE'}, inplace=True)

In [49]:
curr_reg['GAME_DATE'] = pd.to_datetime(curr_reg['GAME_DATE'])
curr_adv['GAME_DATE'] = pd.to_datetime(curr_adv['GAME_DATE'])

In [50]:
curr = curr_reg.merge(
    curr_adv.drop(columns=['W/L', 'MIN', 'Team']),
    on=['MATCHUP', 'GAME_DATE'],
    how='left'
)

In [51]:
curr['SEASON_YEAR'] = '2024-25'

In [52]:
curr.rename(columns={'Team': 'TEAM_ABBREVIATION'}, inplace=True)

In [53]:
percent_cols = [col for col in curr.columns if col.endswith('%')]

curr[percent_cols] = curr[percent_cols] / 100

In [54]:
curr.rename(columns={
    '3P%': 'FG3_PCT',
    '3PA': 'FG3A',
    '3PM': 'FG3M',
    'FG%': 'FG_PCT',
    'FT%': 'FT_PCT',
    'W/L': 'WL',
    '+/-': 'PLUS_MINUS'
}, inplace=True)

In [55]:
curr = curr.sort_values(by=['TEAM_ABBREVIATION', 'GAME_DATE'])

In [56]:
numerics = curr.select_dtypes(include=['int', 'float']).drop(columns='MIN').columns
numerics

Index(['PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PLUS_MINUS', 'OffRtg', 'DefRtg', 'NetRtg', 'AST%', 'AST/TO',
       'ASTRatio', 'OREB%', 'DREB%', 'REB%', 'TOV%', 'eFG%', 'TS%', 'PACE',
       'PIE'],
      dtype='object')

In [57]:
for col in numerics:
    curr[f"{col}_AVG_5"] = curr.groupby('TEAM_ABBREVIATION')[col].transform(lambda x: x.shift(1).rolling(window=5, min_periods=5).mean())

for col in numerics:
    curr[f"{col}_AVG_10"] = curr.groupby('TEAM_ABBREVIATION')[col].transform(lambda x: x.shift(1).rolling(window=10, min_periods=10).mean())

In [58]:
pd.set_option('display.max_columns', None)
curr[(curr['TEAM_ABBREVIATION'] == 'WAS')].shape

(60, 105)

In [59]:
momentum = {}

for col in numerics:
    momentum[f"{col}_MOMENTUM"] = (curr[f"{col}_AVG_5"] - curr[f"{col}_AVG_10"]) / curr[f"{col}_AVG_10"]

In [60]:
momentum_df = pd.DataFrame(momentum)
curr = pd.concat([curr, momentum_df], axis=1)

In [61]:
curr.head(11)

,TEAM_ABBREVIATION,MATCHUP,GAME_DATE,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,OffRtg,DefRtg,NetRtg,AST%,AST/TO,ASTRatio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE,SEASON_YEAR,PTS_AVG_5,FGM_AVG_5,FGA_AVG_5,FG_PCT_AVG_5,FG3M_AVG_5,FG3A_AVG_5,FG3_PCT_AVG_5,FTM_AVG_5,FTA_AVG_5,FT_PCT_AVG_5,OREB_AVG_5,DREB_AVG_5,REB_AVG_5,AST_AVG_5,STL_AVG_5,BLK_AVG_5,TOV_AVG_5,PF_AVG_5,PLUS_MINUS_AVG_5,OffRtg_AVG_5,DefRtg_AVG_5,NetRtg_AVG_5,AST%_AVG_5,AST/TO_AVG_5,ASTRatio_AVG_5,OREB%_AVG_5,DREB%_AVG_5,REB%_AVG_5,TOV%_AVG_5,eFG%_AVG_5,TS%_AVG_5,PACE_AVG_5,PIE_AVG_5,PTS_AVG_10,FGM_AVG_10,FGA_AVG_10,FG_PCT_AVG_10,FG3M_AVG_10,FG3A_AVG_10,FG3_PCT_AVG_10,FTM_AVG_10,FTA_AVG_10,FT_PCT_AVG_10,OREB_AVG_10,DREB_AVG_10,REB_AVG_10,AST_AVG_10,STL_AVG_10,BLK_AVG_10,TOV_AVG_10,PF_AVG_10,PLUS_MINUS_AVG_10,OffRtg_AVG_10,DefRtg_AVG_10,NetRtg_AVG_10,AST%_AVG_10,AST/TO_AVG_10,ASTRatio_AVG_10,OREB%_AVG_10,DREB%_AVG_10,REB%_AVG_10,TOV%_AVG_10,eFG%_AVG_10,TS%_AVG_10,PACE_AVG_10,PIE_AVG_10,PTS_MOMENTUM,FGM_MOMENTUM,FGA_MOMENTUM,FG_PCT_MOMENTUM,FG3M_MOMENTUM,FG3A_MOMENTUM,FG3_PCT_MOMENTUM,FTM_MOMENTUM,FTA_MOMENTUM,FT_PCT_MOMENTUM,OREB_MOMENTUM,DREB_MOMENTUM,REB_MOMENTUM,AST_MOMENTUM,STL_MOMENTUM,BLK_MOMENTUM,TOV_MOMENTUM,PF_MOMENTUM,PLUS_MINUS_MOMENTUM,OffRtg_MOMENTUM,DefRtg_MOMENTUM,NetRtg_MOMENTUM,AST%_MOMENTUM,AST/TO_MOMENTUM,ASTRatio_MOMENTUM,OREB%_MOMENTUM,DREB%_MOMENTUM,REB%_MOMENTUM,TOV%_MOMENTUM,eFG%_MOMENTUM,TS%_MOMENTUM,PACE_MOMENTUM,PIE_MOMENTUM
1813,ATL,ATL vs. BKN,2024-10-23,W,240,120,39,80,0.488,9,28,0.321,33,46,0.717,12,33,45,25,12,9,16,20,4,114.3,110.5,3.8,0.641,1.56,17.4,0.304,0.679,0.510,0.152,0.544,0.599,105.0,58.9,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1785,ATL,ATL vs. CHA,2024-10-25,W,240,125,39,81,0.481,14,38,0.368,33,38,0.868,7,32,39,25,11,9,14,28,5,119.0,116.5,2.5,0.641,1.79,17.6,0.267,0.567,0.438,0.133,0.568,0.640,104.0,57.7,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1740,ATL,ATL @ OKC,2024-10-27,L,240,104,36,91,0.396,10,31,0.323,22,29,0.759,17,32,49,24,7,4,20,23,-24,103.0,125.5,-22.5,0.667,1.20,16.2,0.383,0.694,0.523,0.198,0.451,0.501,101.5,36.4,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1717,ATL,ATL vs. WAS,2024-10-28,L,240,119,39,81,0.481,15,40,0.375,26,36,0.722,6,33,39,32,12,7,16,22,-2,112.3,115.2,-3.0,0.821,2.00,21.6,0.222,0.725,0.490,0.151,0.574,0.614,105.5,54.2,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705,ATL,ATL @ WAS,2024-10-30,L,240,120,45,95,0.474,12,39,0.308,18,21,0.857,12,29,41,28,10,3,17,19,-13,113.2,125.5,-12.3,0.622,1.65,18.5,0.360,0.744,0.538,0.160,0

In [62]:
curr.shape

(1818, 138)

In [63]:
curr.WL = curr.WL.map({'W': 1, 'L': 0})

In [64]:
win_cumsum = curr.groupby('TEAM_ABBREVIATION')['WL'].cumsum().shift(1)
game_count = curr.groupby('TEAM_ABBREVIATION')['WL'].cumcount()

curr['WIN_PCT'] = win_cumsum / game_count

In [65]:
home = curr[~curr.MATCHUP.str.contains("@")]
away = curr[curr.MATCHUP.str.contains("@")]

In [66]:
duplicates = ['SEASON_YEAR', 'MIN']
merge_on = ['GAME_DATE', 'MATCHUP']
away = away.drop(columns=duplicates)
home = home.rename(columns=lambda col: f"H_{col}" if col not in duplicates + merge_on else col)
away = away.rename(columns=lambda col: f"A_{col}" if col not in merge_on else col)

In [67]:
away = away.drop(columns=['A_WL'])

In [68]:
modified = []

for game in away.MATCHUP:
    modified.append(game[-3:] + " vs. " + game[:3])

In [69]:
away['MATCHUP'] = modified

In [70]:
curr_merged = pd.merge(home, away, on=['MATCHUP', 'GAME_DATE'], how='inner')
curr_merged.head()

,H_TEAM_ABBREVIATION,MATCHUP,GAME_DATE,H_WL,MIN,H_PTS,H_FGM,H_FGA,H_FG_PCT,H_FG3M,H_FG3A,H_FG3_PCT,H_FTM,H_FTA,H_FT_PCT,H_OREB,H_DREB,H_REB,H_AST,H_STL,H_BLK,H_TOV,H_PF,H_PLUS_MINUS,H_OffRtg,H_DefRtg,H_NetRtg,H_AST%,H_AST/TO,H_ASTRatio,H_OREB%,H_DREB%,H_REB%,H_TOV%,H_eFG%,H_TS%,H_PACE,H_PIE,SEASON_YEAR,H_PTS_AVG_5,H_FGM_AVG_5,H_FGA_AVG_5,H_FG_PCT_AVG_5,H_FG3M_AVG_5,H_FG3A_AVG_5,H_FG3_PCT_AVG_5,H_FTM_AVG_5,H_FTA_AVG_5,H_FT_PCT_AVG_5,H_OREB_AVG_5,H_DREB_AVG_5,H_REB_AVG_5,H_AST_AVG_5,H_STL_AVG_5,H_BLK_AVG_5,H_TOV_AVG_5,H_PF_AVG_5,H_PLUS_MINUS_AVG_5,H_OffRtg_AVG_5,H_DefRtg_AVG_5,H_NetRtg_AVG_5,H_AST%_AVG_5,H_AST/TO_AVG_5,H_ASTRatio_AVG_5,H_OREB%_AVG_5,H_DREB%_AVG_5,H_REB%_AVG_5,H_TOV%_AVG_5,H_eFG%_AVG_5,H_TS%_AVG_5,H_PACE_AVG_5,H_PIE_AVG_5,H_PTS_AVG_10,H_FGM_AVG_10,H_FGA_AVG_10,H_FG_PCT_AVG_10,H_FG3M_AVG_10,H_FG3A_AVG_10,H_FG3_PCT_AVG_10,H_FTM_AVG_10,H_FTA_AVG_10,H_FT_PCT_AVG_10,H_OREB_AVG_10,H_DREB_AVG_10,H_REB_AVG_10,H_AST_AVG_10,H_STL_AVG_10,H_BLK_AVG_10,H_TOV_AVG_10,H_PF_AVG_10,H_PLUS_MINUS_AVG_10,H_OffRtg_AVG_10,H_DefRtg_AVG_10,H_NetRtg_AVG_10,H_AST%_AVG_10,H_AST/TO_AVG_10,H_ASTRatio_AVG_10,H_OREB%_AVG_10,H_DREB%_AVG_10,H_REB%_AVG_10,H_TOV%_AVG_10,H_eFG%_AVG_10,H_TS%_AVG_10,H_PACE_AVG_10,H_PIE_AVG_10,H_PTS_MOMENTUM,H_FGM_MOMENTUM,H_FGA_MOMENTUM,H_FG_PCT_MOMENTUM,H_FG3M_MOMENTUM,H_FG3A_MOMENTUM,H_FG3_PCT_MOMENTUM,H_FTM_MOMENTUM,H_FTA_MOMENTUM,H_FT_PCT_MOMENTUM,H_OREB_MOMENTUM,H_DREB_MOMENTUM,H_REB_MOMENTUM,H_AST_MOMENTUM,H_STL_MOMENTUM,H_BLK_MOMENTUM,H_TOV_MOMENTUM,H_PF_MOMENTUM,H_PLUS_MINUS_MOMENTUM,H_OffRtg_MOMENTUM,H_DefRtg_MOMENTUM,H_NetRtg_MOMENTUM,H_AST%_MOMENTUM,H_AST/TO_MOMENTUM,H_ASTRatio_MOMENTUM,H_OREB%_MOMENTUM,H_DREB%_MOMENTUM,H_REB%_MOMENTUM,H_TOV%_MOMENTUM,H_eFG%_MOMENTUM,H_TS%_MOMENTUM,H_PACE_MOMENTUM,H_PIE_MOMENTUM,H_WIN_PCT,A_TEAM_ABBREVIATION,A_PTS,A_FGM,A_FGA,A_FG_PCT,A_FG3M,A_FG3A,A_FG3_PCT,A_FTM,A_FTA,A_FT_PCT,A_OREB,A_DREB,A_REB,A_AST,A_STL,A_BLK,A_TOV,A_PF,A_PLUS_MINUS,A_OffRtg,A_DefRtg,A_NetRtg,A_AST%,A_AST/TO,A_ASTRatio,A_OREB%,A_DREB%,A_REB%,A_TOV%,A_eFG%,A_TS%,A_PACE,A_PIE,A_PTS_AVG_5,A_FGM_AVG_5,A_FGA_AVG_5,A_FG_PCT_AVG_5,A_FG3M_AVG_5,A_FG3A_AVG_5,A_FG3_PCT_AVG_5,A_FTM_AVG_5,A_FTA_AVG_5,A_FT_PCT_AVG_5,A_OREB_AVG_5,A_DREB_AVG_5,A_REB_AVG_5,A_AST_AVG_5,A_STL_AVG_5,A_BLK_AVG_5,A_TOV_AVG_5,A_PF_AVG_5,A_PLUS_MINUS_AVG_5,A_OffRtg_AVG_5,A_DefRtg_AVG_5,A_NetRtg_AVG_5,A_AST%_AVG_5,A_AST/TO_AVG_5,A_ASTRatio_AVG_5,A_OREB%_AVG_5,A_DREB%_AVG_5,A_REB%_AVG_5,A_TOV%_AVG_5,A_eFG%_AVG_5,A_TS%_AVG_5,A_PACE_AVG_5,A_PIE_AVG_5,A_PTS_AVG_10,A_FGM_AVG_10,A_FGA_AVG_10,A_FG_PCT_AVG_10,A_FG3M_AVG_10,A_FG3A_AVG_10,A_FG3_PCT_AVG_10,A_FTM_AVG_10,A_FTA_AVG_10,A_FT_PCT_AVG_10,A_OREB_AVG_10,A_DREB_AVG_10,A_REB_AVG_10,A_AST_AVG_10,A_STL_AVG_10,A_BLK_AVG_10,A_TOV_AVG_10,A_PF_AVG_10,A_PLUS_MINUS_AVG_10,A_OffRtg_AVG_10,A_DefRtg_AVG_10,A_NetRtg_AVG_10,A_AST%_AVG_10,A_AST/TO_AVG_10,A_ASTRatio_AVG_10,A_OREB%_AVG_10,A_DREB%_AVG_10,A_REB%_AVG_10,A_TOV%_AVG_10,A_eFG%_AVG_10,A_TS%_AVG_10,A_PACE_AVG_10,A_PIE_AVG_10,A_PTS_MOMENTUM,A_FGM_MOMENTUM,A_FGA_MOMENTUM,A_FG_PCT_MOMENTUM,A_FG3M_MOMENTUM,A_FG3A_MOMENTUM,A_FG3_PCT_MOMENTUM,A_FTM_MOMENTUM,A_FTA_MOMENTUM,A_FT_PCT_MOMENTUM,A_OREB_MOMENTUM,A_DREB_MOMENTUM,A_REB_MOMENTUM,A_AST_MOMENTUM,A_STL_MOMENTUM,A_BLK_MOMENTUM,A_TOV_MOMENTUM,A_PF_MOMENTUM,A_PLUS_MINUS_MOMENTUM,A_OffRtg_MOMENTUM,A_DefRtg_MOMENTUM,A_NetRtg_MOMENTUM,A_AST%_MOMENTUM,A_AST/TO_MOMENTUM,A_ASTRatio_MOMENTUM,A_OREB%_MOMENTUM,A_DREB%_MOMENTUM,A_REB%_MOMENTUM,A_TOV%_MOMENTUM,A_eFG%_MOMENTUM,A_TS%_MOMENTUM,A_PACE_MOMENTUM,A_PIE_MOMENTUM,A_WIN_PCT
0,ATL,ATL vs. BKN,2024-10-23,1,240,120,39,80,0.488,9,28,0.321,33,46,0.717,12,33,45,25,12,9,16,20,4,114.3,110.5,3.8,0.641,1.56,17.4,0.304,0.679,0.510,0.152,0.544,0.599,105.0,58.9,2024-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [71]:
front = ['SEASON_YEAR', 'GAME_DATE', 'MIN', 'H_TEAM_ABBREVIATION', 'A_TEAM_ABBREVIATION', 'MATCHUP']
back = [col for col in curr_merged.columns if col not in front]

curr_merged = curr_merged[front + back].sort_values(by='GAME_DATE')
curr_merged

,SEASON_YEAR,GAME_DATE,MIN,H_TEAM_ABBREVIATION,A_TEAM_ABBREVIATION,MATCHUP,H_WL,H_PTS,H_FGM,H_FGA,H_FG_PCT,H_FG3M,H_FG3A,H_FG3_PCT,H_FTM,H_FTA,H_FT_PCT,H_OREB,H_DREB,H_REB,H_AST,H_STL,H_BLK,H_TOV,H_PF,H_PLUS_MINUS,H_OffRtg,H_DefRtg,H_NetRtg,H_AST%,H_AST/TO,H_ASTRatio,H_OREB%,H_DREB%,H_REB%,H_TOV%,H_eFG%,H_TS%,H_PACE,H_PIE,H_PTS_AVG_5,H_FGM_AVG_5,H_FGA_AVG_5,H_FG_PCT_AVG_5,H_FG3M_AVG_5,H_FG3A_AVG_5,H_FG3_PCT_AVG_5,H_FTM_AVG_5,H_FTA_AVG_5,H_FT_PCT_AVG_5,H_OREB_AVG_5,H_DREB_AVG_5,H_REB_AVG_5,H_AST_AVG_5,H_STL_AVG_5,H_BLK_AVG_5,H_TOV_AVG_5,H_PF_AVG_5,H_PLUS_MINUS_AVG_5,H_OffRtg_AVG_5,H_DefRtg_AVG_5,H_NetRtg_AVG_5,H_AST%_AVG_5,H_AST/TO_AVG_5,H_ASTRatio_AVG_5,H_OREB%_AVG_5,H_DREB%_AVG_5,H_REB%_AVG_5,H_TOV%_AVG_5,H_eFG%_AVG_5,H_TS%_AVG_5,H_PACE_AVG_5,H_PIE_AVG_5,H_PTS_AVG_10,H_FGM_AVG_10,H_FGA_AVG_10,H_FG_PCT_AVG_10,H_FG3M_AVG_10,H_FG3A_AVG_10,H_FG3_PCT_AVG_10,H_FTM_AVG_10,H_FTA_AVG_10,H_FT_PCT_AVG_10,H_OREB_AVG_10,H_DREB_AVG_10,H_REB_AVG_10,H_AST_AVG_10,H_STL_AVG_10,H_BLK_AVG_10,H_TOV_AVG_10,H_PF_AVG_10,H_PLUS_MINUS_AVG_10,H_OffRtg_AVG_10,H_DefRtg_AVG_10,H_NetRtg_AVG_10,H_AST%_AVG_10,H_AST/TO_AVG_10,H_ASTRatio_AVG_10,H_OREB%_AVG_10,H_DREB%_AVG_10,H_REB%_AVG_10,H_TOV%_AVG_10,H_eFG%_AVG_10,H_TS%_AVG_10,H_PACE_AVG_10,H_PIE_AVG_10,H_PTS_MOMENTUM,H_FGM_MOMENTUM,H_FGA_MOMENTUM,H_FG_PCT_MOMENTUM,H_FG3M_MOMENTUM,H_FG3A_MOMENTUM,H_FG3_PCT_MOMENTUM,H_FTM_MOMENTUM,H_FTA_MOMENTUM,H_FT_PCT_MOMENTUM,H_OREB_MOMENTUM,H_DREB_MOMENTUM,H_REB_MOMENTUM,H_AST_MOMENTUM,H_STL_MOMENTUM,H_BLK_MOMENTUM,H_TOV_MOMENTUM,H_PF_MOMENTUM,H_PLUS_MINUS_MOMENTUM,H_OffRtg_MOMENTUM,H_DefRtg_MOMENTUM,H_NetRtg_MOMENTUM,H_AST%_MOMENTUM,H_AST/TO_MOMENTUM,H_ASTRatio_MOMENTUM,H_OREB%_MOMENTUM,H_DREB%_MOMENTUM,H_REB%_MOMENTUM,H_TOV%_MOMENTUM,H_eFG%_MOMENTUM,H_TS%_MOMENTUM,H_PACE_MOMENTUM,H_PIE_MOMENTUM,H_WIN_PCT,A_PTS,A_FGM,A_FGA,A_FG_PCT,A_FG3M,A_FG3A,A_FG3_PCT,A_FTM,A_FTA,A_FT_PCT,A_OREB,A_DREB,A_REB,A_AST,A_STL,A_BLK,A_TOV,A_PF,A_PLUS_MINUS,A_OffRtg,A_DefRtg,A_NetRtg,A_AST%,A_AST/TO,A_ASTRatio,A_OREB%,A_DREB%,A_REB%,A_TOV%,A_eFG%,A_TS%,A_PACE,A_PIE,A_PTS_AVG_5,A_FGM_AVG_5,A_FGA_AVG_5,A_FG_PCT_AVG_5,A_FG3M_AVG_5,A_FG3A_AVG_5,A_FG3_PCT_AVG_5,A_FTM_AVG_5,A_FTA_AVG_5,A_FT_PCT_AVG_5,A_OREB_AVG_5,A_DREB_AVG_5,A_REB_AVG_5,A_AST_AVG_5,A_STL_AVG_5,A_BLK_AVG_5,A_TOV_AVG_5,A_PF_AVG_5,A_PLUS_MINUS_AVG_5,A_OffRtg_AVG_5,A_DefRtg_AVG_5,A_NetRtg_AVG_5,A_AST%_AVG_5,A_AST/TO_AVG_5,A_ASTRatio_AVG_5,A_OREB%_AVG_5,A_DREB%_AVG_5,A_REB%_AVG_5,A_TOV%_AVG_5,A_eFG%_AVG_5,A_TS%_AVG_5,A_PACE_AVG_5,A_PIE_AVG_5,A_PTS_AVG_10,A_FGM_AVG_10,A_FGA_AVG_10,A_FG_PCT_AVG_10,A_FG3M_AVG_10,A_FG3A_AVG_10,A_FG3_PCT_AVG_10,A_FTM_AVG_10,A_FTA_AVG_10,A_FT_PCT_AVG_10,A_OREB_AVG_10,A_DREB_AVG_10,A_REB_AVG_10,A_AST_AVG_10,A_STL_AVG_10,A_BLK_AVG_10,A_TOV_AVG_10,A_PF_AVG_10,A_PLUS_MINUS_AVG_10,A_OffRtg_AVG_10,A_DefRtg_AVG_10,A_NetRtg_AVG_10,A_AST%_AVG_10,A_AST/TO_AVG_10,A_ASTRatio_AVG_10,A_OREB%_AVG_10,A_DREB%_AVG_10,A_REB%_AVG_10,A_TOV%_AVG_10,A_eFG%_AVG_10,A_TS%_AVG_10,A_PACE_AVG_10,A_PIE_AVG_10,A_PTS_MOMENTUM,A_FGM_MOMENTUM,A_FGA_MOMENTUM,A_FG_PCT_MOMENTUM,A_FG3M_MOMENTUM,A_FG3A_MOMENTUM,A_FG3_PCT_MOMENTUM,A_FTM_MOMENTUM,A_FTA_MOMENTUM,A_FT_PCT_MOMENTUM,A_OREB_MOMENTUM,A_DREB_MOMENTUM,A_REB_MOMENTUM,A_AST_MOMENTUM,A_STL_MOMENTUM,A_BLK_MOMENTUM,A_TOV_MOMENTUM,A_PF_MOMENTUM,A_PLUS_MINUS_MOMENTUM,A_OffRtg_MOMENTUM,A_DefRtg_MOMENTUM,A_NetRtg_MOMENTUM,A_AST%_MOMENTUM,A_AST/TO_MOMENTUM,A_ASTRatio_MOMENTUM,A_OREB%_MOMENTUM,A_DREB%_MOMENTUM,A_REB%_MOMENTUM,A_TOV%_MOMENTUM,A_eFG%_MOMENTUM,A_TS%_MOMENTUM,A_PACE_MOMENTUM,A_PIE_MOMENTUM,A_WIN_PCT
390,2024-25,2024-10-22,240,LAL,MIN,LAL vs. MIN,1,110,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,7,115.8,109.6,6.2,0.524,3.14,15.9,0.375,0.630,0.500,0.074,0.468,0.519,94.5,57.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [72]:
curr_merged['SPREAD'] = curr_merged.H_PTS - curr_merged.A_PTS

In [73]:
curr_merged['TOTAL'] = curr_merged.H_PTS + curr_merged.A_PTS

In [74]:
curr_merged['OREB'] = curr_merged.H_OREB + curr_merged.A_OREB

In [75]:
curr_merged.shape

(904, 276)

In [76]:
past = pd.read_csv('ARMGameStats.csv')
past.shape

(5829, 298)

In [77]:
past.H_WL = past.H_WL.map({'W': 1, 'L': 0})

In [78]:
curr_merged.to_csv('Current.csv')

In [79]:
overlap = set(past.columns) - set(curr_merged.columns)
overlap

{'A_BLKA',
 'A_BLKA_AVG_10',
 'A_BLKA_AVG_5',
 'A_BLKA_MOMENTUM',
 'A_PFD',
 'A_PFD_AVG_10',
 'A_PFD_AVG_5',
 'A_PFD_MOMENTUM',
 'A_TEAM_ID',
 'A_TEAM_NAME',
 'GAME_ID',
 'H_BLKA',
 'H_BLKA_AVG_10',
 'H_BLKA_AVG_5',
 'H_BLKA_MOMENTUM',
 'H_PFD',
 'H_PFD_AVG_10',
 'H_PFD_AVG_5',
 'H_PFD_MOMENTUM',
 'H_TEAM_ID',
 'H_TEAM_NAME',
 'Unnamed: 0'}

In [80]:
final = pd.concat([past, curr_merged], ignore_index=True).drop(columns=['MIN', 'Unnamed: 0'])

In [81]:
features = [col for col in final.columns if "AVG" in col or "MOMENTUM" in col or "WIN_PCT" in col]
info = [col for col in final.columns if col not in features]

features = [i for i in features if i not in overlap]
len(features)

200

In [82]:
final = final[info + features]
final

,SEASON_YEAR,GAME_ID,GAME_DATE,H_TEAM_ID,H_TEAM_ABBREVIATION,H_TEAM_NAME,A_TEAM_ID,A_TEAM_ABBREVIATION,A_TEAM_NAME,MATCHUP,H_WL,H_FGM,H_FGA,H_FG_PCT,H_FG3M,H_FG3A,H_FG3_PCT,H_FTM,H_FTA,H_FT_PCT,H_OREB,H_DREB,H_REB,H_AST,H_TOV,H_STL,H_BLK,H_BLKA,H_PF,H_PFD,H_PTS,H_PLUS_MINUS,A_FGM,A_FGA,A_FG_PCT,A_FG3M,A_FG3A,A_FG3_PCT,A_FTM,A_FTA,A_FT_PCT,A_OREB,A_DREB,A_REB,A_AST,A_TOV,A_STL,A_BLK,A_BLKA,A_PF,A_PFD,A_PTS,A_PLUS_MINUS,SPREAD,TOTAL,OREB,H_OffRtg,H_DefRtg,H_NetRtg,H_AST%,H_AST/TO,H_ASTRatio,H_OREB%,H_DREB%,H_REB%,H_TOV%,H_eFG%,H_TS%,H_PACE,H_PIE,A_OffRtg,A_DefRtg,A_NetRtg,A_AST%,A_AST/TO,A_ASTRatio,A_OREB%,A_DREB%,A_REB%,A_TOV%,A_eFG%,A_TS%,A_PACE,A_PIE,H_FGM_AVG_5,H_FGA_AVG_5,H_FG_PCT_AVG_5,H_FG3M_AVG_5,H_FG3A_AVG_5,H_FG3_PCT_AVG_5,H_FTM_AVG_5,H_FTA_AVG_5,H_FT_PCT_AVG_5,H_OREB_AVG_5,H_DREB_AVG_5,H_REB_AVG_5,H_AST_AVG_5,H_TOV_AVG_5,H_STL_AVG_5,H_BLK_AVG_5,H_PF_AVG_5,H_PTS_AVG_5,H_PLUS_MINUS_AVG_5,H_OffRtg_AVG_5,H_DefRtg_AVG_5,H_NetRtg_AVG_5,H_AST%_AVG_5,H_AST/TO_AVG_5,H_ASTRatio_AVG_5,H_OREB%_AVG_5,H_DREB%_AVG_5,H_REB%_AVG_5,H_TOV%_AVG_5,H_eFG%_AVG_5,H_TS%_AVG_5,H_PACE_AVG_5,H_PIE_AVG_5,H_FGM_AVG_10,H_FGA_AVG_10,H_FG_PCT_AVG_10,H_FG3M_AVG_10,H_FG3A_AVG_10,H_FG3_PCT_AVG_10,H_FTM_AVG_10,H_FTA_AVG_10,H_FT_PCT_AVG_10,H_OREB_AVG_10,H_DREB_AVG_10,H_REB_AVG_10,H_AST_AVG_10,H_TOV_AVG_10,H_STL_AVG_10,H_BLK_AVG_10,H_PF_AVG_10,H_PTS_AVG_10,H_PLUS_MINUS_AVG_10,H_OffRtg_AVG_10,H_DefRtg_AVG_10,H_NetRtg_AVG_10,H_AST%_AVG_10,H_AST/TO_AVG_10,H_ASTRatio_AVG_10,H_OREB%_AVG_10,H_DREB%_AVG_10,H_REB%_AVG_10,H_TOV%_AVG_10,H_eFG%_AVG_10,H_TS%_AVG_10,H_PACE_AVG_10,H_PIE_AVG_10,A_FGM_AVG_5,A_FGA_AVG_5,A_FG_PCT_AVG_5,A_FG3M_AVG_5,A_FG3A_AVG_5,A_FG3_PCT_AVG_5,A_FTM_AVG_5,A_FTA_AVG_5,A_FT_PCT_AVG_5,A_OREB_AVG_5,A_DREB_AVG_5,A_REB_AVG_5,A_AST_AVG_5,A_TOV_AVG_5,A_STL_AVG_5,A_BLK_AVG_5,A_PF_AVG_5,A_PTS_AVG_5,A_PLUS_MINUS_AVG_5,A_OffRtg_AVG_5,A_DefRtg_AVG_5,A_NetRtg_AVG_5,A_AST%_AVG_5,A_AST/TO_AVG_5,A_ASTRatio_AVG_5,A_OREB%_AVG_5,A_DREB%_AVG_5,A_REB%_AVG_5,A_TOV%_AVG_5,A_eFG%_AVG_5,A_TS%_AVG_5,A_PACE_AVG_5,A_PIE_AVG_5,A_FGM_AVG_10,A_FGA_AVG_10,A_FG_PCT_AVG_10,A_FG3M_AVG_10,A_FG3A_AVG_10,A_FG3_PCT_AVG_10,A_FTM_AVG_10,A_FTA_AVG_10,A_FT_PCT_AVG_10,A_OREB_AVG_10,A_DREB_AVG_10,A_REB_AVG_10,A_AST_AVG_10,A_TOV_AVG_10,A_STL_AVG_10,A_BLK_AVG_10,A_PF_AVG_10,A_PTS_AVG_10,A_PLUS_MINUS_AVG_10,A_OffRtg_AVG_10,A_DefRtg_AVG_10,A_NetRtg_AVG_10,A_AST%_AVG_10,A_AST/TO_AVG_10,A_ASTRatio_AVG_10,A_OREB%_AVG_10,A_DREB%_AVG_10,A_REB%_AVG_10,A_TOV%_AVG_10,A_eFG%_AVG_10,A_TS%_AVG_10,A_PACE_AVG_10,A_PIE_AVG_10,H_FGM_MOMENTUM,H_FGA_MOMENTUM,H_FG_PCT_MOMENTUM,H_FG3M_MOMENTUM,H_FG3A_MOMENTUM,H_FG3_PCT_MOMENTUM,H_FTM_MOMENTUM,H_FTA_MOMENTUM,H_FT_PCT_MOMENTUM,H_OREB_MOMENTUM,H_DREB_MOMENTUM,H_REB_MOMENTUM,H_AST_MOMENTUM,H_TOV_MOMENTUM,H_STL_MOMENTUM,H_BLK_MOMENTUM,H_PF_MOMENTUM,H_PTS_MOMENTUM,H_PLUS_MINUS_MOMENTUM,H_OffRtg_MOMENTUM,H_DefRtg_MOMENTUM,H_NetRtg_MOMENTUM,H_AST%_MOMENTUM,H_AST/TO_MOMENTUM,H_ASTRatio_MOMENTUM,H_OREB%_MOMENTUM,H_DREB%_MOMENTUM,H_REB%_MOMENTUM,H_TOV%_MOMENTUM,H_eFG%_MOMENTUM,H_TS%_MOMENTUM,H_PACE_MOMENTUM,H_PIE_MOMENTUM,A_FGM_MOMENTUM,A_FGA_MOMENTUM,A_FG_PCT_MOMENTUM,A_FG3M_MOMENTUM,A_FG3A_MOMENTUM,A_FG3_PCT_MOMENTUM,A_FTM_MOMENTUM,A_FTA_MOMENTUM,A_FT_PCT_MOMENTUM,A_OREB_MOMENTUM,A_DREB_MOMENTUM,A_REB_MOMENTUM,A_AST_MOMENTUM,A_TOV_MOMENTUM,A_STL_MOMENTUM,A_BLK_MOMENTUM,A_PF_MOMENTUM,A_PTS_MOMENTUM,A_PLUS_MINUS_MOMENTUM,A_OffRtg_MOMENTUM,A_DefRtg_MOMENTUM,A_NetRtg_MOMENTUM,A_AST%_MOMENTUM,A_AST/TO_MOMENTUM,A_ASTRatio_MOMENTUM,A_OREB%_MOMENTUM,A_DREB%_MOMENTUM,A_REB%_MOMENTUM,A_TOV%_MOMENTUM,A_eFG%_MOMENTUM,A_TS%_MOMENTUM,A_PACE_MOMENTUM,A_PIE_MOMENTUM,H_WIN_PCT,A_WIN_PCT
0,2019-20,21900001.0,2019-10-22,1.610613e+09,TOR,Toronto Raptors,1.610613e+09,NOP,New Orleans Pelicans,TOR vs. NOP,1,42,103,0.408,14,40,0.350,32,38,0.842,16,41,57,23,17.0,7,3,9.0,24,34.0,130,8.0,43,102,0.422,19,45,0.422,17,20,0.850,16,37,53,30,19.0,4,9,3.0,34,24.0,122,-8.0,8,252,32,113.0,108.9,4.1,0.548,1.35,14.2,0.400,0.729,0.556,0.148,0.476,0.543,102.79,53.1,108.9,113.0,-4.1,0.698,1.58,19.0,0.271,0.

In [84]:
final.to_csv('FINAL.csv')

In [85]:
final.columns.tolist()

['SEASON_YEAR',
 'GAME_ID',
 'GAME_DATE',
 'H_TEAM_ID',
 'H_TEAM_ABBREVIATION',
 'H_TEAM_NAME',
 'A_TEAM_ID',
 'A_TEAM_ABBREVIATION',
 'A_TEAM_NAME',
 'MATCHUP',
 'H_WL',
 'H_FGM',
 'H_FGA',
 'H_FG_PCT',
 'H_FG3M',
 'H_FG3A',
 'H_FG3_PCT',
 'H_FTM',
 'H_FTA',
 'H_FT_PCT',
 'H_OREB',
 'H_DREB',
 'H_REB',
 'H_AST',
 'H_TOV',
 'H_STL',
 'H_BLK',
 'H_BLKA',
 'H_PF',
 'H_PFD',
 'H_PTS',
 'H_PLUS_MINUS',
 'A_FGM',
 'A_FGA',
 'A_FG_PCT',
 'A_FG3M',
 'A_FG3A',
 'A_FG3_PCT',
 'A_FTM',
 'A_FTA',
 'A_FT_PCT',
 'A_OREB',
 'A_DREB',
 'A_REB',
 'A_AST',
 'A_TOV',
 'A_STL',
 'A_BLK',
 'A_BLKA',
 'A_PF',
 'A_PFD',
 'A_PTS',
 'A_PLUS_MINUS',
 'SPREAD',
 'TOTAL',
 'OREB',
 'H_OffRtg',
 'H_DefRtg',
 'H_NetRtg',
 'H_AST%',
 'H_AST/TO',
 'H_ASTRatio',
 'H_OREB%',
 'H_DREB%',
 'H_REB%',
 'H_TOV%',
 'H_eFG%',
 'H_TS%',
 'H_PACE',
 'H_PIE',
 'A_OffRtg',
 'A_DefRtg',
 'A_NetRtg',
 'A_AST%',
 'A_AST/TO',
 'A_ASTRatio',
 'A_OREB%',
 'A_DREB%',
 'A_REB%',
 'A_TOV%',
 'A_eFG%',
 'A_TS%',
 'A_PACE',
 'A_PIE',
 'H_F